In [2]:
"""The data stored in pickle format with version: 0.7.5 (python 3.7).
The type of graph object：networkx.classes.multidigraph.MultiDiGraph
Numbers of nodes: 2973489
Numbers of edges: 13551303
Average degree:   4.5574
Nodes' features：
    // The label. 1 means fishing mark node, otherwise 0.
    G.nodes[nodeName]['isp']；

Edges' features:
    G[node1][node2][0]['amount']        // The amount mount of the transaction.
    G[node1][node2][0]['timestamp']     // The timestamp of the transaction.				
							
* Notes * 
"""
# Quick start tutorial.

import pickle 
import networkx as nx

def load_pickle(fname):
    with open(fname, 'rb') as f:
        return pickle.load(f)

G = load_pickle('/mnt/data/ethereum-phishing-transaction-network/MulDiGraph.pkl')

In [2]:
# Traversal nodes:
for idx, nd in enumerate(nx.nodes(G)):
    print(nd)
    print(G.nodes[nd]['isp'])
    break

0x1f1e784a61a8ca0a90250bcd2170696655b28a21
0


In [30]:
import time
# Travelsal edges:
min_time = 1e20
max_time = 0
for ind, edge in enumerate(nx.edges(G)):
    (u, v) = edge
    eg = G[u][v][0]
    amo, tim = eg['amount'], eg['timestamp']
    min_time = min(min_time, tim)
    max_time = max(max_time, tim)
min_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(min_time))
max_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(max_time))
print(min_time, max_time)

2015-08-07 07:01:09 2019-01-19 09:32:09


In [4]:
import pandas as pd
# get raw transactions
transactions_path = '/mnt/data/ethereum-phishing-transaction-network/transactions.csv'
df = pd.read_csv(transactions_path)
print(df.head())
print(df.size)

                                                hash  nonce  \
0  0x5c504ed432cb51138bcf09aa5e8a410dd4a1e204ef84...      0   
1  0x19f1df2c7ee6b464720ad28e903aeda1a5ad8780afc2...      0   
2  0x9e6e19637bb625a8ff3d052b7c2fe57dc78c55a15d25...      0   
3  0xcb9378977089c773c074045b20ede2cdcc3a6ff562f4...      0   
4  0x570ce19176bd0002b04a9179309129bbdaf0c4252ffe...      0   

                                          block_hash  block_number  \
0  0x4e3a3754410177e6937ef1f84bba68ea139e8d1a2258...         46147   
1  0x5793f91c9fa8f824d8ed77fc1687dddcf334da81c68b...         46169   
2  0xf4a537e8e2233149929a9b6964c9aced6ee95f42131a...         46170   
3  0x47ec6a0c3467850cf88112c212c262819de6f1d084d3...         46194   
4  0xe6fb31b12d06b5a70f420a28b4e034bcd152abc2d603...         46205   

   transaction_index                                from_address  \
0                  0  0xa1e4380a3b1f749673e270229993ee55f35663b4   
1                  0  0xbd08e0cddec097db7901ea819a3d1fd9de8951a2

In [5]:
c = 0
def count_matching_edges(edge_list, df):
    # Convert the edge list to a set for faster lookup
    edge_set = set(edge_list)
    
    # Create a boolean mask for matching edges
    mask = df.apply(lambda row: (row['from_address'], row['to_address']) in edge_set, axis=1)
    
    
    # Count the number of True values in the mask
    count = mask.sum()
    
    return count

edge_list = list(G.edges())
matching_count = count_matching_edges(edge_list, df)
print(f"Number of matching edges: {matching_count}")
print(f"Number of total edges: {len(edge_list)}")
print(f"Percentage of matching edges: {matching_count/len(edge_list)}")


Number of matching edges: 119438
Number of total edges: 13551303
Percentage of matching edges: 0.008813764993668875


In [34]:
from google.cloud import bigquery

client = bigquery.Client(project='cse3000')
lol = ['0x1f1e784a61a8ca0a90250bcd2170696655b28a21']
lil = ['0x1266f8b9e4dffc9e2f719bf51713f7e714516861']
# Perform a query.
QUERY = (
    # 'SELECT to_address, from_address FROM `bigquery-public-data.crypto_ethereum.transactions` '
    # f'WHERE to_address = {lol[0]}'
    # f'AND from_address = {lil[0]}'
    #f'WHERE to_address IN UNNEST({lol})'
    #f'AND from_address IN UNNEST({lil})'
    'SELECT * FROM `bigquery-public-data.crypto_ethereum.transactions`'
    'WHERE block_timestamp BETWEEN TIMESTAMP("2015-08-07") AND TIMESTAMP("2019-01-19")'
    'ORDER BY block_timestamp ASC LIMIT 10'

    # 'LIMIT 100'
)
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row)

BadRequest: 400 Syntax error: Expected end of input but got identifier "ASCLIMIT" at [1:173]; reason: invalidQuery, location: query, message: Syntax error: Expected end of input but got identifier "ASCLIMIT" at [1:173]

Location: US
Job ID: 32254874-aff3-4278-9ca6-7472a14915ef
